# Домашнее задание № 3. Исправление опечаток

In [1]:
!pip install textdistance

In [2]:
import textdistance

In [3]:
!pip install razdel

In [4]:
import re
from string import punctuation
from collections import Counter
punctuation += "«»—…“”"
punct = set(punctuation)

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_distances

In [6]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [8]:
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

In [9]:
def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])

    # вся эффективноть берется из того, что мы сразу считаем близость
    # 1 вектора ко всей матрице (словам в словаре)
    # считать по отдельности циклом было бы дольше
    # вместо одного вектора может даже целая матрица
    # тогда считаться в итоге будет ещё быстрее

    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn]

    return [(id2word[top], similarities[top]) for top in topn]

In [10]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()

    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word)

    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    min_distance = closest[0][1] # минимальное расстояние редактирования
    # находим слова с минимальным расстоянием редактирования:
    filtered_candidates = [cand for cand in closest if cand[1] == min_distance]
    # сортируем слова с минимальным расстоянием редактирования по вероятности:
    filtered_candidates = sorted(filtered_candidates, key=lambda x: P(x[0]), reverse=True)

    return filtered_candidates


N = sum(vocab.values())

def P(word, N=N): # преобразование слов в вероятности
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1


In [11]:
get_closest_hybrid_match('сонце', X, vec)

[('солнце', 0.8333333333333334)]

## 2.  Symspell (7 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  

2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!)

3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  

4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

1) Словарь правильных слов:

In [20]:
# Оставим тот же корпус, что и в первом задании
corpus = open('data/wiki_data.txt', encoding='utf8').read()

# словарь правильных слов
vocab = Counter(re.findall('\w+', corpus.lower()))

# всего слов в словаре
N = sum(vocab.values())

# функция, которая выдает нам вероятности слова
def P(word, N=N):
  return vocab[word] / N

In [21]:
vocab

Counter({'новостройка': 10,
         'нижегородская': 32,
         'область': 3387,
         'новостро': 1,
         'йка': 6,
         'сельский': 451,
         'посёлок': 818,
         'в': 267296,
         'дивеевском': 9,
         'районе': 3790,
         'нижегородской': 150,
         'области': 5963,
         'входит': 1360,
         'состав': 3887,
         'сатисского': 10,
         'сельсовета': 526,
         'расположен': 1150,
         '12': 3083,
         '5': 6819,
         'км': 3721,
         'к': 22383,
         'югу': 295,
         'от': 18527,
         'села': 1182,
         'дивеева': 4,
         'и': 147115,
         '1': 10736,
         'западу': 497,
         'города': 3660,
         'сарова': 3,
         'на': 81926,
         'правом': 394,
         'берегу': 1205,
         'реки': 2108,
         'вичкинза': 8,
         'правый': 152,
         'приток': 197,
         'сатис': 39,
         'окружён': 31,
         'смешанными': 9,
         'лесами': 75,
         'с

2) Словарь удалений:

In [22]:
# функция, которая генерирует все возможные удаления одного символа
def get_deletions(word, max_deletions=1):
  splits = [(word[:i], word[i:]) for i in range(len(word) + 1)] # список разбиений слова на левую и правую части
  deletes = [L + R[1:] for L, R in splits if R] # удаляем нулевой (первый) символ из правой части
  return deletes


# словарь удалений
def build_deletion_dict(vocab):
  deletion_dict = {}

  for word in vocab:
    for deletion in get_deletions(word):
      if deletion not in deletion_dict:
        deletion_dict[deletion] = []
      deletion_dict[deletion].append(word)
  return deletion_dict

In [23]:
deletion_dict = build_deletion_dict(vocab)

In [24]:
deletion_dict

{'овостройка': ['новостройка'],
 'нвостройка': ['новостройка'],
 'ноостройка': ['новостройка'],
 'новстройка': ['новостройка'],
 'новотройка': ['новостройка'],
 'новосройка': ['новостройка'],
 'новостойка': ['новостройка'],
 'новострйка': ['новостройка'],
 'новострока': ['новостройка'],
 'новостройа': ['новостройка'],
 'новостройк': ['новостройка', 'новостройки'],
 'ижегородская': ['нижегородская'],
 'нжегородская': ['нижегородская'],
 'ниегородская': ['нижегородская'],
 'нижгородская': ['нижегородская'],
 'нижеородская': ['нижегородская'],
 'нижегродская': ['нижегородская'],
 'нижегоодская': ['нижегородская'],
 'нижегордская': ['нижегородская'],
 'нижегороская': ['нижегородская'],
 'нижегородкая': ['нижегородская'],
 'нижегородсая': ['нижегородская'],
 'нижегородскя': ['нижегородская'],
 'нижегородска': ['нижегородская'],
 'бласть': ['область'],
 'оласть': ['область'],
 'обасть': ['область'],
 'облсть': ['область'],
 'облать': ['область'],
 'облась': ['область'],
 'област': ['область'

3) Выбор исправлений

In [25]:
# функция, которая ищет ближайшее совпадение
def get_closest_match(word, deletion_dict, vocab, topn=10):

  # генерируем все варианты удаления
  deletions = get_deletions(word)
  candidates = []

  # находим кандидатов в словаре удалений
  for deletion in deletions:
    if deletion in deletion_dict:
      candidates.extend(deletion_dict[deletion])

  # находим наиболее вероятных кандидатов
  candidates = Counter(candidates)
  ranked_candidates = sorted(candidates.items(), key=lambda x: P(x[0]), reverse=True)

  return ranked_candidates[:topn]

Пробуем алгоритм на слове с опечаткой "концерк"

In [26]:
get_closest_match('концерк', deletion_dict, vocab, topn=10)

[('концерт', 1), ('концерн', 1)]

Оценка кода, используются следующие метрики:
1) процент правильных слов;
2) процент исправленных ошибок
3) процент ошибочно исправленных правильных слов

In [29]:
!pip install razdel tqdm

In [35]:
# библиотека для отслеживания прогресса
from tqdm import tqdm

In [38]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

In [43]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()

    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]

    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]

    assert len(tokens_1) == len(tokens_2)

    return list(zip(tokens_1, tokens_2))

In [44]:
correct = 0
total = 0

total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

cashed = {}

for i in tqdm(range(len(true))):
    # сопоставляем слова из правильного и ошибочного текста для каждого предложения
    word_pairs = align_words(true[i], bad[i])

    for pair in word_pairs:
        if pair[1] not in cashed:
          # если слово с ошибкой еще не встречалось, ищем исправление
          predicted = get_closest_match(pair[1], deletion_dict, vocab, topn=1)
          # добавляем исправление, если оно есть, иначе оставляем слово с ошибкой
          predicted_word = predicted[0][0] if predicted else pair[1]
          # кешируем слово
          cashed[pair[1]] = predicted_word
        else:
          predicted_word = cashed[pair[1]]


        if predicted_word == pair[0]:
            correct += 1
        total += 1

        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] !=  predicted_word:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == predicted_word:
                mistaken_fixed += 1

100%|██████████| 915/915 [00:01<00:00, 520.75it/s] 


In [47]:
print("Процент правильных слов:", correct/total)
print("Процент исправленных ошибок:", mistaken_fixed/total_mistaken)
print("Процент ошибочно исправленных правильных слов:", correct_broken/total_correct)

Процент правильных слов: 0.43491745872936466
Процент исправленных ошибок: 0.2127329192546584
Процент ошибочно исправленных правильных слов: 0.5322154588262318
